# Entorno

In [ ]:
# en una T4 de Colab
# !pip install unsloth

In [3]:
import torch

print(torch.__version__)
print(torch.version.cuda)
torch.cuda.is_available()

2.7.0+cu126
12.6


True

# Dataset

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

df = pd.read_csv("https://raw.githubusercontent.com/eduardofc/data/refs/heads/main/opl_programming_language_2.csv")
print(df.shape)
df.head()

,question,response
0,What does OPL stand for in the OPL programming...,OPL is short for Open Programming Language
1,Which company developed the OPL programmung la...,Psion Ltd created OPL for the Psion Organiser
2,Which was the original name for the OPL progra...,The OPL language was originally named Organise...
3,Who created the OPL programming language?,The OPL language was designed by Colly Myers a...
4,What was the first implementation of OPL?,The first implementation of the OPL programmin...


In [16]:
from datasets import Dataset

ds = Dataset.from_pandas(df)
ds

Dataset({
    features: ['question', 'response'],
    num_rows: 43
})

In [17]:
ds[18]

{'question': 'in OPL, + is used for addition, - is used for subtraction, / is used for division, and * is used for multiplication. for example: LOCAL calculation%=((10+20-6)*3)/2',
 'response': 'PROC calculations:\n\tLOCAL num1% = 10\n\tLOCAL num2% = 5\n\n\tPRINT "Addition:", num1% + num2%\n\tPRINT "Subtraction:", num1% - num2%\n\tPRINT "Multiplication:", num1% * num2%\n\tPRINT "Division:", num1% / num2%\n\n\tGET\nENDP'}

# Model

In [ ]:
from unsloth import FastLanguageModel

max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.6.2: Fast Llama patching. Transformers: 4.52.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

# Before fine tuning

In [11]:
prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Response:
{}"""

In [7]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    prompt.format(
        "Write a hello world program in the OPL programming language.", # instruction
        "", # response
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nWrite a hello world program in the OPL programming language.\n\n### Input:\n\n\n### Response:\n```OPL\nprint("Hello World!")\n```<|end_of_text|>']

In [12]:
tokenizer.eos_token

'<|end_of_text|>'

In [103]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth: Already have LoRA adapters! We shall skip this step.


In [96]:
ds2[18]

{'question': 'in OPL, + is used for addition, - is used for subtraction, / is used for division, and * is used for multiplication. for example: LOCAL calculation%=((10+20-6)*3)/2',
 'response': 'PROC calculations:\n\tLOCAL num1% = 10\n\tLOCAL num2% = 5\n\n\tPRINT "Addition:", num1% + num2%\n\tPRINT "Subtraction:", num1% - num2%\n\tPRINT "Multiplication:", num1% * num2%\n\tPRINT "Division:", num1% / num2%\n\n\tGET\nENDP'}

In [ ]:
def formatting_prompts_func(x):
    instruction = x["question"]
    output      = x["response"]
    x['text'] = prompt.format(instruction, output) + tokenizer.eos_token
    return x


ds3 = ds2.map(formatting_prompts_func).remove_columns(['question', 'response'])

In [102]:
ds3[18]

{'text': 'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nin OPL, + is used for addition, - is used for subtraction, / is used for division, and * is used for multiplication. for example: LOCAL calculation%=((10+20-6)*3)/2\n\n### Response:\nPROC calculations:\n\tLOCAL num1% = 10\n\tLOCAL num2% = 5\n\n\tPRINT "Addition:", num1% + num2%\n\tPRINT "Subtraction:", num1% - num2%\n\tPRINT "Multiplication:", num1% * num2%\n\tPRINT "Division:", num1% / num2%\n\n\tGET\nENDP<|end_of_text|>'}

# Fine tuning

In [106]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = ds3,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"]:   0%|          | 0/43 [00:00<?, ? examples/s]

In [107]:
%%time
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 43 | Num Epochs = 10 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.374000
2,2.483400
3,2.980500
4,2.235300
5,2.082600
6,2.569000
7,1.809700
8,1.560900
9,1.119900
10,1.117700


CPU times: user 3min 39s, sys: 39.3 s, total: 4min 18s
Wall time: 4min 54s


TrainOutput(global_step=60, training_loss=0.6383178925762574, metrics={'train_runtime': 292.0266, 'train_samples_per_second': 1.644, 'train_steps_per_second': 0.205, 'total_flos': 2621238468673536.0, 'train_loss': 0.6383178925762574})

# After fine tuning

In [108]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    prompt.format(
        "Write a hello world program in the OPL programming language.", # instruction
        "", # response
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nWrite a hello world program in the OPL programming language.\n\n### Response:\nPROC main:\n\tPRINT "Hello World"\n\tGET\nENDP<|end_of_text|>']